<a href="https://colab.research.google.com/github/jasmis1229/midterm_new/blob/main/%ED%8A%B8%EB%A0%8C%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ✅ CBOW 실험 초기화

# 1. 라이브러리 불러오기
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import re
import random
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

# 2. 데이터 불러오기
df = pd.read_csv('/content/신조어_스타일_패턴_완성본_UTF8SIG.csv')  # 파일명은 환경에 맞게 수정

In [2]:
# ✅ CBOW 데이터 정제

# 신조어 앞 번호 제거 함수
def clean_word(word):
    word = str(word)
    return re.sub(r'^\d+\.', '', word).strip()

# 클린 신조어 컬럼 생성
df['클린 신조어'] = df['신조어'].apply(clean_word)

In [3]:
# ✅ CBOW 설명 토크나이징

# 간단 토크나이징 함수
def simple_tokenize(text):
    text = re.sub(r"[^\w\s]", "", str(text))  # 특수문자 제거
    tokens = text.strip().split()
    tokens = [token for token in tokens if len(token) > 1]  # 한 글자 제거
    return tokens

# 설명 토크나이징 적용
descriptions = df['설명'].tolist()
tokenized_descriptions = [simple_tokenize(desc) for desc in descriptions]

In [4]:
# ✅ CBOW 어휘 사전 구축

vocab = set()
for tokens in tokenized_descriptions:
    vocab.update(tokens)

# 단어 ↔ 인덱스 매핑
vocab = {word: idx for idx, word in enumerate(vocab)}
idx_to_word = {idx: word for word, idx in vocab.items()}
vocab_size = len(vocab)

In [5]:
# ✅ CBOW 학습 데이터셋 생성

window_size = 2
context_center_pairs = []

for tokens in tokenized_descriptions:
    for idx, center_word in enumerate(tokens):
        context = []
        for i in range(idx - window_size, idx + window_size + 1):
            if i != idx and 0 <= i < len(tokens):
                context.append(vocab[tokens[i]])
        if context:
            context_center_pairs.append((context, vocab[center_word]))

In [6]:
# ✅ CBOW Dataset 클래스 정의

class CBOWDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        context, center = self.data[idx]
        return torch.tensor(context), torch.tensor(center)

# DataLoader 생성
dataset = CBOWDataset(context_center_pairs)
train_loader = DataLoader(dataset, batch_size=128, shuffle=True, collate_fn=lambda batch: list(zip(*batch)))

In [7]:
# ✅ CBOW 모델 클래스 정의

class CBOWModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=100):
        super(CBOWModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)

    def forward(self, contexts):
        embeds = []
        for context in contexts:
            embed = self.embeddings(context)
            embeds.append(embed.mean(dim=0))
        embeds = torch.stack(embeds)
        out = self.linear(embeds)
        return out

In [8]:
# ✅ CBOW 모델 초기화

model = CBOWModel(vocab_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
# ✅ CBOW 학습 루프

num_epochs = 20

for epoch in range(num_epochs):
    total_loss = 0
    for contexts, centers in train_loader:
        outputs = model(contexts)
        loss = criterion(outputs, torch.tensor(centers))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}')

Epoch [1/20], Loss: 102.8183
Epoch [2/20], Loss: 99.6956
Epoch [3/20], Loss: 96.8455
Epoch [4/20], Loss: 94.1754
Epoch [5/20], Loss: 91.3577
Epoch [6/20], Loss: 88.4945
Epoch [7/20], Loss: 85.8633
Epoch [8/20], Loss: 83.0899
Epoch [9/20], Loss: 80.3510
Epoch [10/20], Loss: 77.5348
Epoch [11/20], Loss: 74.8449
Epoch [12/20], Loss: 72.0077
Epoch [13/20], Loss: 69.2950
Epoch [14/20], Loss: 66.4486
Epoch [15/20], Loss: 63.9486
Epoch [16/20], Loss: 61.0662
Epoch [17/20], Loss: 58.3412
Epoch [18/20], Loss: 55.7727
Epoch [19/20], Loss: 53.1345
Epoch [20/20], Loss: 50.5563


# **여기까지가 CBOW 학습**

In [11]:
# ✅ CBOW 문맥 예측 테스트

model.eval()

random_idx = random.randint(0, len(context_center_pairs) - 1)
random_context, true_center = context_center_pairs[random_idx]

context_tensor = torch.tensor([random_context])
predicted_logits = model(context_tensor)
predicted_probs = F.softmax(predicted_logits, dim=1)
top5 = torch.topk(predicted_probs, 5)

print("\n[CBOW 예측 테스트 결과]")
print(f"문맥(Context): {[idx_to_word[idx] for idx in random_context]}")
print(f"정답(Center): {idx_to_word[true_center]}")

print("\nTop-5 예측 단어:")
for i in range(5):
    idx = top5.indices[0][i].item()
    prob = top5.values[0][i].item()
    print(f"{i+1}. {idx_to_word[idx]} (확률: {prob:.4f})")


[CBOW 예측 테스트 결과]
문맥(Context): ['여행을', '가든', '하든', '일않하고']
정답(Center): 뭔일을

Top-5 예측 단어:
1. 뭔일을 (확률: 0.0135)
2. 손가락은 (확률: 0.0085)
3. 하든 (확률: 0.0037)
4. 않도록 (확률: 0.0029)
5. 게시물 (확률: 0.0026)


In [12]:
# ✅ CBOW 예측 최적화

model.eval()

random_idx = random.randint(0, len(context_center_pairs) - 1)
random_context, true_center = context_center_pairs[random_idx]

with torch.no_grad():
    context_tensor = torch.tensor([random_context])
    predicted_logits = model(context_tensor)
    predicted_probs = F.softmax(predicted_logits, dim=1)
    top5 = torch.topk(predicted_probs, 5)

print("\n[CBOW 예측 테스트 결과 개선]")
print(f"문맥(Context): {[idx_to_word[idx] for idx in random_context]}")
print(f"정답(Center): {idx_to_word[true_center]}")

print("\nTop-5 예측 단어:")
for i in range(5):
    idx = top5.indices[0][i].item()
    prob = top5.values[0][i].item()
    print(f"{i+1}. {idx_to_word[idx]} (확률: {prob:.4f})")


[CBOW 예측 테스트 결과 개선]
문맥(Context): ['싶어서', '눈알이']
정답(Center): 꼿힐때

Top-5 예측 단어:
1. 꼿힐때 (확률: 0.0488)
2. 물건 (확률: 0.0101)
3. 사고 (확률: 0.0075)
4. 뜸들어 (확률: 0.0072)
5. 눈알이 (확률: 0.0072)


In [14]:
# ✅ CBOW 스타일 기반 미래 신조어 생성

sample_rows = df.sample(3, random_state=random.randint(0,10000))

print("\n[미래 신조어 스타일 예측 및 생성]")

selected_words = []
combined_styles = set()

for idx, row in sample_rows.iterrows():
    word = row['클린 신조어']
    style = row['스타일 패턴']
    print(f"기존 신조어: {word}")
    print(f"스타일 패턴: {style}\n")
    selected_words.append(word)
    combined_styles.update(style.split(', '))

def safe_pick(word, pos):
    if len(word) == 1:
        return word
    if pos == 'start':
        return word[0]
    elif pos == 'middle':
        return word[len(word)//2]
    elif pos == 'end':
        return word[-1]
    else:
        return random.choice(word)

new_created_word = (
    safe_pick(selected_words[0], 'start') +
    safe_pick(selected_words[1], 'middle') +
    safe_pick(selected_words[2], 'end')
)

print(f"👉 생성된 미래 신조어: {new_created_word}")
print(f"👉 예상되는 스타일 패턴: {', '.join(list(combined_styles))}")


[미래 신조어 스타일 예측 및 생성]
기존 신조어: 비담
스타일 패턴: 리듬감

기존 신조어: 자라족
스타일 패턴: 리듬감+감정성

기존 신조어: 선리후감
스타일 패턴: 리듬감+감정성

👉 생성된 미래 신조어: 비라감
👉 예상되는 스타일 패턴: 리듬감+감정성, 리듬감


# 첫 시도 코드

In [ ]:
# ✅ Colab용 CBOW 미래 신조어 스타일 예측

# 1. 라이브러리 불러오기
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import re
from sklearn.model_selection import train_test_split

# 2. 데이터 불러오기 (Colab에 업로드한 파일 직접 경로로 읽기)
df = pd.read_csv('/content/신조어_데이터셋_최종본.csv')

# 3. 간단 토크나이징 함수
def simple_tokenize(text):
    text = re.sub(r"[^\w\s]", "", str(text))  # 특수문자 제거
    tokens = text.strip().split()
    tokens = [token for token in tokens if len(token) > 1]  # 한 글자 제거
    return tokens

# 4. 데이터 전처리: 설명을 토크나이징
descriptions = df['설명'].tolist()
tokenized_descriptions = [simple_tokenize(desc) for desc in descriptions]

# 5. 어휘 사전 구축
vocab = set()
for tokens in tokenized_descriptions:
    vocab.update(tokens)
vocab = {word: idx for idx, word in enumerate(vocab)}
idx_to_word = {idx: word for word, idx in vocab.items()}
vocab_size = len(vocab)

# 6. CBOW 학습용 데이터셋 생성
window_size = 2
context_center_pairs = []

for tokens in tokenized_descriptions:
    for idx, center_word in enumerate(tokens):
        context = []
        for i in range(idx - window_size, idx + window_size + 1):
            if i != idx and 0 <= i < len(tokens):
                context.append(vocab[tokens[i]])
        if context:
            context_center_pairs.append((context, vocab[center_word]))

# 7. Dataset 클래스 정의
class CBOWDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        context, center = self.data[idx]
        return torch.tensor(context), torch.tensor(center)

# 8. DataLoader 생성
dataset = CBOWDataset(context_center_pairs)
train_loader = DataLoader(dataset, batch_size=128, shuffle=True, collate_fn=lambda batch: list(zip(*batch)))

# 9. CBOW 모델 정의
class CBOWModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=100):
        super(CBOWModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)

    def forward(self, contexts):
        embeds = []
        for context in contexts:
            embed = self.embeddings(context)
            embeds.append(embed.mean(dim=0))
        embeds = torch.stack(embeds)
        out = self.linear(embeds)
        return out

# 10. 모델 초기화
model = CBOWModel(vocab_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 11. 학습 루프
num_epochs = 20

for epoch in range(num_epochs):
    total_loss = 0
    for contexts, centers in train_loader:
        outputs = model(contexts)
        loss = criterion(outputs, torch.tensor(centers))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}')

# 12. 학습 완료!

Epoch [1/20], Loss: 102.7254
Epoch [2/20], Loss: 99.6482
Epoch [3/20], Loss: 96.8595
Epoch [4/20], Loss: 94.0647
Epoch [5/20], Loss: 91.2805
Epoch [6/20], Loss: 88.6131
Epoch [7/20], Loss: 85.7855
Epoch [8/20], Loss: 83.0773
Epoch [9/20], Loss: 80.2598
Epoch [10/20], Loss: 77.5700
Epoch [11/20], Loss: 74.9174
Epoch [12/20], Loss: 72.1179
Epoch [13/20], Loss: 69.3065
Epoch [14/20], Loss: 66.5300
Epoch [15/20], Loss: 63.8406
Epoch [16/20], Loss: 61.1367
Epoch [17/20], Loss: 58.4678
Epoch [18/20], Loss: 55.9640
Epoch [19/20], Loss: 53.1117
Epoch [20/20], Loss: 50.5708


# 개선

In [ ]:
# ✅ Colab용 CBOW 미래 신조어 스타일 예측 기본 코드 (예측 테스트 추가)

# 1. 라이브러리 불러오기
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import re
import random
import torch.nn.functional as F

# 2. 데이터 불러오기 (Colab에 업로드한 파일 직접 경로로 읽기)
df = pd.read_csv('/content/신조어_데이터셋_최종본_랜덤라벨링.csv')

# 3. 간단 토크나이징 함수
def simple_tokenize(text):
    text = re.sub(r"[^\w\s]", "", str(text))  # 특수문자 제거
    tokens = text.strip().split()
    tokens = [token for token in tokens if len(token) > 1]  # 한 글자 제거
    return tokens

# 4. 데이터 전처리: 설명을 토크나이징
descriptions = df['설명'].tolist()
tokenized_descriptions = [simple_tokenize(desc) for desc in descriptions]

# 5. 어휘 사전 구축
vocab = set()
for tokens in tokenized_descriptions:
    vocab.update(tokens)
vocab = {word: idx for idx, word in enumerate(vocab)}
idx_to_word = {idx: word for word, idx in vocab.items()}
vocab_size = len(vocab)

# 6. CBOW 학습용 데이터셋 생성
window_size = 2
context_center_pairs = []

for tokens in tokenized_descriptions:
    for idx, center_word in enumerate(tokens):
        context = []
        for i in range(idx - window_size, idx + window_size + 1):
            if i != idx and 0 <= i < len(tokens):
                context.append(vocab[tokens[i]])
        if context:
            context_center_pairs.append((context, vocab[center_word]))

# 7. Dataset 클래스 정의
class CBOWDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        context, center = self.data[idx]
        return torch.tensor(context), torch.tensor(center)

# 8. DataLoader 생성
dataset = CBOWDataset(context_center_pairs)
train_loader = DataLoader(dataset, batch_size=128, shuffle=True, collate_fn=lambda batch: list(zip(*batch)))

# 9. CBOW 모델 정의
class CBOWModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=100):
        super(CBOWModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)

    def forward(self, contexts):
        embeds = []
        for context in contexts:
            embed = self.embeddings(context)
            embeds.append(embed.mean(dim=0))
        embeds = torch.stack(embeds)
        out = self.linear(embeds)
        return out

# 10. 모델 초기화
model = CBOWModel(vocab_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 11. 학습 루프
num_epochs = 20

for epoch in range(num_epochs):
    total_loss = 0
    for contexts, centers in train_loader:
        outputs = model(contexts)
        loss = criterion(outputs, torch.tensor(centers))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}')

# 12. 예측 테스트
model.eval()

# 랜덤 context-center 하나 뽑기
random_idx = random.randint(0, len(context_center_pairs) - 1)
random_context, true_center = context_center_pairs[random_idx]

# 예측 수행
context_tensor = torch.tensor([random_context])
predicted_logits = model(context_tensor)
predicted_probs = F.softmax(predicted_logits, dim=1)
top5 = torch.topk(predicted_probs, 5)

print("\n[예측 테스트 결과]")
print(f"문맥(Context): {[idx_to_word[idx] for idx in random_context]}")
print(f"정답(Center): {idx_to_word[true_center]}")

print("\nTop-5 예측 단어:")
for i in range(5):
    idx = top5.indices[0][i].item()
    prob = top5.values[0][i].item()
    print(f"{i+1}. {idx_to_word[idx]} (확률: {prob:.4f})")

# 13. 학습 및 테스트 완료!

Epoch [1/20], Loss: 102.6710
Epoch [2/20], Loss: 99.6654
Epoch [3/20], Loss: 96.8958
Epoch [4/20], Loss: 94.1144
Epoch [5/20], Loss: 91.4124
Epoch [6/20], Loss: 88.6321
Epoch [7/20], Loss: 85.8246
Epoch [8/20], Loss: 83.1512
Epoch [9/20], Loss: 80.4493
Epoch [10/20], Loss: 77.7554
Epoch [11/20], Loss: 74.9357
Epoch [12/20], Loss: 72.1357
Epoch [13/20], Loss: 69.4429
Epoch [14/20], Loss: 66.6867
Epoch [15/20], Loss: 63.9483
Epoch [16/20], Loss: 61.3349
Epoch [17/20], Loss: 58.5307
Epoch [18/20], Loss: 55.7705
Epoch [19/20], Loss: 53.3817
Epoch [20/20], Loss: 50.6909

[예측 테스트 결과]
문맥(Context): ['마스크를', '벗었을때', '모습이']
정답(Center): 썼을때와

Top-5 예측 단어:
1. 썼을때와 (확률: 0.0142)
2. 벗었을때 (확률: 0.0033)
3. 마스크를 (확률: 0.0032)
4. 더럽다 (확률: 0.0031)
5. 비슷함을 (확률: 0.0030)


# 스타일 패턴 동일하게 실행(실패)

In [ ]:
# 1. 라이브러리 불러오기
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import re
import random
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

# 2. 데이터 불러오기
df = pd.read_csv('/content/신조어_데이터셋_최종본_랜덤라벨링.csv')

# 3. 번호 제거 함수
def clean_word(word):
    word = str(word)
    return re.sub(r'^\d+\.', '', word).strip()

# 4. 스타일 패턴 생성 함수 (숫자리듬 제거)
def assign_style_in_code(word, desc):
    styles = set()
    if 2 <= len(word) <= 4:
        styles.add('짧음')
    if any(kw in word for kw in ['킹', '존', '갓', '빡', '철컹', '극', '찐', '꿀', '맛탱']):
        styles.add('감정성')
    if re.search(r'(..)\1', word) or '봐' in word or '놈' in word:
        styles.add('리듬감')
    if any(kw in desc for kw in ['현실', '사바사', '복세편살', '편하게', '문화', '진상', '현생']):
        styles.add('현실성')
    if any(kw in desc for kw in ['웃긴', '밈', '웃음', '패러디', '유머', '개그']):
        styles.add('유머성')
    if any(kw in desc for kw in ['폭', '테러', '공격', '비하', '혐오']):
        styles.add('과격성')
    if not styles:
        styles.add('짧음')
    return ', '.join(styles)

# 5. 신조어 클린 + 스타일 추가
df['클린 신조어'] = df['신조어'].apply(clean_word)
df['스타일 패턴'] = df.apply(lambda row: assign_style_in_code(row['클린 신조어'], row['설명']), axis=1)

# 6. 간단 토크나이징
def simple_tokenize(text):
    text = re.sub(r"[^\w\s]", "", str(text))
    tokens = text.strip().split()
    tokens = [token for token in tokens if len(token) > 1]
    return tokens

# 7. 설명 토크나이징
descriptions = df['설명'].tolist()
tokenized_descriptions = [simple_tokenize(desc) for desc in descriptions]

# 8. 어휘 사전 구축
vocab = set()
for tokens in tokenized_descriptions:
    vocab.update(tokens)
vocab = {word: idx for idx, word in enumerate(vocab)}
idx_to_word = {idx: word for word, idx in vocab.items()}
vocab_size = len(vocab)

# 9. CBOW 학습용 데이터셋 생성
window_size = 2
context_center_pairs = []

for tokens in tokenized_descriptions:
    for idx, center_word in enumerate(tokens):
        context = []
        for i in range(idx - window_size, idx + window_size + 1):
            if i != idx and 0 <= i < len(tokens):
                context.append(vocab[tokens[i]])
        if context:
            context_center_pairs.append((context, vocab[center_word]))

# 10. Dataset 클래스 정의
class CBOWDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        context, center = self.data[idx]
        return torch.tensor(context), torch.tensor(center)

# 11. DataLoader 생성
dataset = CBOWDataset(context_center_pairs)
train_loader = DataLoader(dataset, batch_size=128, shuffle=True, collate_fn=lambda batch: list(zip(*batch)))

# 12. CBOW 모델 정의
class CBOWModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=100):
        super(CBOWModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)

    def forward(self, contexts):
        embeds = []
        for context in contexts:
            embed = self.embeddings(context)
            embeds.append(embed.mean(dim=0))
        embeds = torch.stack(embeds)
        out = self.linear(embeds)
        return out

# 13. 모델 초기화
model = CBOWModel(vocab_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 14. CBOW 모델 학습
num_epochs = 20

for epoch in range(num_epochs):
    total_loss = 0
    for contexts, centers in train_loader:
        outputs = model(contexts)
        loss = criterion(outputs, torch.tensor(centers))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}')

# 15. CBOW 문맥 예측 테스트
model.eval()

# 랜덤 context-center 하나 뽑기
random_idx = random.randint(0, len(context_center_pairs) - 1)
random_context, true_center = context_center_pairs[random_idx]

# 예측 수행
context_tensor = torch.tensor([random_context])
predicted_logits = model(context_tensor)
predicted_probs = F.softmax(predicted_logits, dim=1)
top5 = torch.topk(predicted_probs, 5)

print("\n[CBOW 예측 테스트 결과]")
print(f"문맥(Context): {[idx_to_word[idx] for idx in random_context]}")
print(f"정답(Center): {idx_to_word[true_center]}")

print("\nTop-5 예측 단어:")
for i in range(5):
    idx = top5.indices[0][i].item()
    prob = top5.values[0][i].item()
    print(f"{i+1}. {idx_to_word[idx]} (확률: {prob:.4f})")

# 16. 미래 신조어 예측 및 생성

# 3개 랜덤 추출
sample_rows = df.sample(3, random_state=random.randint(0,10000))

print("\n[미래 신조어 스타일 예측 및 새로운 신조어 생성]")
selected_words = []
combined_styles = set()

for idx, row in sample_rows.iterrows():
    word = row['클린 신조어']
    style = row['스타일 패턴']
    print(f"기존 신조어: {word}")
    print(f"스타일 패턴: {style}\n")
    selected_words.append(word)
    combined_styles.update(style.split(', '))

# 신조어 생성 (앞, 중간, 끝 조합)
def safe_pick(word, pos):
    if len(word) == 1:
        return word
    if pos == 'start':
        return word[0]
    elif pos == 'middle':
        return word[len(word)//2]
    elif pos == 'end':
        return word[-1]
    else:
        return random.choice(word)

new_created_word = (
    safe_pick(selected_words[0], 'start') +
    safe_pick(selected_words[1], 'middle') +
    safe_pick(selected_words[2], 'end')
)

print(f"👉 생성된 미래 신조어: {new_created_word}")
print(f"👉 예상되는 스타일 패턴: {', '.join(list(combined_styles))}")

Epoch [1/20], Loss: 103.0230
Epoch [2/20], Loss: 99.9260
Epoch [3/20], Loss: 97.1296
Epoch [4/20], Loss: 94.3229
Epoch [5/20], Loss: 91.5756
Epoch [6/20], Loss: 88.7025
Epoch [7/20], Loss: 86.0574
Epoch [8/20], Loss: 83.2617
Epoch [9/20], Loss: 80.4505
Epoch [10/20], Loss: 77.7155
Epoch [11/20], Loss: 74.8336
Epoch [12/20], Loss: 72.2546
Epoch [13/20], Loss: 69.4565
Epoch [14/20], Loss: 66.6130
Epoch [15/20], Loss: 63.9407
Epoch [16/20], Loss: 61.2313
Epoch [17/20], Loss: 58.5000
Epoch [18/20], Loss: 55.9597
Epoch [19/20], Loss: 53.3505
Epoch [20/20], Loss: 50.5186

[CBOW 예측 테스트 결과]
문맥(Context): ['닉네임']
정답(Center): 차별

Top-5 예측 단어:
1. 차별 (확률: 0.0729)
2. 너무 (확률: 0.0090)
3. 젊은이들 (확률: 0.0066)
4. 부담만 (확률: 0.0059)
5. 기냥 (확률: 0.0053)

[미래 신조어 스타일 예측 및 새로운 신조어 생성]
기존 신조어: 빵프
스타일 패턴: 짧음

기존 신조어: 꾸에엑
스타일 패턴: 짧음

기존 신조어: 사바사
스타일 패턴: 짧음

👉 생성된 미래 신조어: 빵에사
👉 예상되는 스타일 패턴: 짧음


# 최종본

In [15]:
import pandas as pd
import random
import re

# 1. 데이터 불러오기
df = pd.read_csv('/content/신조어_스타일_패턴_완성본_UTF8SIG.csv')

# 2. 신조어 번호 제거 함수
def clean_word(word):
    word = str(word)
    return re.sub(r'^\d+\.', '', word).strip()

df['클린 신조어'] = df['신조어'].apply(clean_word)

# 3. 랜덤 1개 신조어 뽑기
first_sample = df.sample(1, random_state=random.randint(0,10000)).iloc[0]
first_word = first_sample['클린 신조어']
first_style = first_sample['스타일 패턴']

print(f"\n[Step 1] 랜덤으로 뽑은 신조어")
print(f"신조어: {first_word}")
print(f"스타일 패턴: {first_style}")

# 4. 비슷한 스타일 패턴 가진 신조어 찾기
# (부분 매칭 포함)
similar_words_df = df[df['스타일 패턴'].str.contains(first_style.split(',')[0].strip())]

# 첫 번째로 뽑은 단어는 제외
similar_words_df = similar_words_df[similar_words_df['클린 신조어'] != first_word]

# 5. 그 중 2개 랜덤으로 추가 추출
additional_samples = similar_words_df.sample(2, random_state=random.randint(0,10000))

selected_words = [first_word] + additional_samples['클린 신조어'].tolist()
selected_styles = [first_style] + additional_samples['스타일 패턴'].tolist()

print(f"\n[Step 2] 비슷한 스타일 패턴을 가진 추가 신조어 2개 뽑기")
for word, style in zip(selected_words[1:], selected_styles[1:]):
    print(f"신조어: {word}, 스타일 패턴: {style}")

# 6. 신조어 생성 로직
def safe_pick(word, pos):
    if len(word) == 1:
        return word
    if pos == 'start':
        return word[0]
    elif pos == 'middle':
        return word[len(word)//2]
    elif pos == 'end':
        return word[-1]
    else:
        return random.choice(word)

new_created_word = (
    safe_pick(selected_words[0], 'start') +
    safe_pick(selected_words[1], 'middle') +
    safe_pick(selected_words[2], 'end')
)

# 7. 최종 출력
print(f"\n[Step 3] 👉 3개 단어로 생성된 미래 신조어: {new_created_word}")
combined_styles = set()
for style in selected_styles:
    combined_styles.update(style.split(', '))
print(f"👉 예상되는 스타일 패턴: {', '.join(list(combined_styles))}")



[Step 1] 랜덤으로 뽑은 신조어
신조어: 핑프
스타일 패턴: 음성적표현

[Step 2] 비슷한 스타일 패턴을 가진 추가 신조어 2개 뽑기
신조어: 얀데레, 스타일 패턴: 음성적표현
신조어: 팩폭, 스타일 패턴: 음성적표현

[Step 3] 👉 3개 단어로 생성된 미래 신조어: 핑데폭
👉 예상되는 스타일 패턴: 음성적표현


# 다음은 TF-IDF 유행 가능성 예측

In [16]:
# ✅ TF-IDF 실험 초기화

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import re
import random

# 데이터 불러오기
train_df = pd.read_csv('/content/신조어_데이터셋_유행.csv')
predict_df = pd.read_csv('/content/신조어_데이터셋_예측.csv')

In [17]:
# ✅ TF-IDF 데이터 정제

def clean_word(word):
    word = str(word)
    return re.sub(r'^\d+\.', '', word).strip()

train_df['클린 신조어'] = train_df['신조어'].apply(clean_word)
predict_df['클린 신조어'] = predict_df['신조어'].apply(clean_word)

In [18]:
# ✅ TF-IDF 벡터화

vectorizer = TfidfVectorizer(max_features=300)
X_train = vectorizer.fit_transform(train_df['클린 신조어'].fillna(''))
labels = train_df['유행여부']

In [20]:
# ✅ 로지스틱 회귀 모델 학습

model = LogisticRegression()
model.fit(X_train, labels)

LogisticRegression()

In [21]:
# ✅ 예측 데이터 벡터 변환

X_predict = vectorizer.transform(predict_df['클린 신조어'].fillna(''))

In [24]:
# ✅ 유행 여부 예측

predict_preds = model.predict(X_predict)

for word, pred in zip(predict_df['클린 신조어'], predict_preds):
    print(f"신조어: {word} -> 예측 결과: {'유행할 것' if pred == 1 else '비유행할 것'}")

신조어: 정누마 -> 예측 결과: 비유행할 것
신조어: 미친년 -> 예측 결과: 비유행할 것
신조어: 미친넘 -> 예측 결과: 비유행할 것
신조어: 발컨 -> 예측 결과: 비유행할 것
신조어: 억텐 -> 예측 결과: 비유행할 것
신조어: 중꺽마 -> 예측 결과: 비유행할 것
신조어: 갓생 -> 예측 결과: 비유행할 것
신조어: 이선좌 -> 예측 결과: 비유행할 것
신조어: 네카라쿠배당토 -> 예측 결과: 비유행할 것
신조어: 막나귀 -> 예측 결과: 비유행할 것
신조어: 섹시푸드 -> 예측 결과: 비유행할 것
신조어: 가면비 -> 예측 결과: 비유행할 것
신조어: 추구미 -> 예측 결과: 비유행할 것
신조어: 안남미 -> 예측 결과: 비유행할 것
신조어: 익속 -> 예측 결과: 비유행할 것
신조어: 캘박 -> 예측 결과: 비유행할 것
신조어: 분깨미 -> 예측 결과: 비유행할 것
신조어: 돼지런 -> 예측 결과: 비유행할 것
신조어: 꾸에엑 -> 예측 결과: 비유행할 것
신조어: 감다살 -> 예측 결과: 비유행할 것
신조어: 감다뒤 -> 예측 결과: 비유행할 것
신조어: 종노플예 -> 예측 결과: 비유행할 것
신조어: 싹싹 김치 -> 예측 결과: 비유행할 것
신조어: 일세스코 -> 예측 결과: 비유행할 것
신조어: 한플루언서 -> 예측 결과: 비유행할 것
신조어: 위쑤시개 -> 예측 결과: 비유행할 것
신조어: 밥플릭스 -> 예측 결과: 비유행할 것
신조어: 랜선생님=랜선상님=랜선생=랜위인 -> 예측 결과: 비유행할 것
신조어: 수발새끼 -> 예측 결과: 비유행할 것
신조어: 수발년 -> 예측 결과: 비유행할 것
신조어: 무지컬 -> 예측 결과: 비유행할 것
신조어: 쌍무지컬 -> 예측 결과: 비유행할 것
신조어: 손절미 -> 예측 결과: 비유행할 것
신조어: 테무 인간 -> 예측 결과: 비유행할 것
신조어: 느좋 -> 예측 결과: 비유행할 것
신조어: 깊생 -> 예측 결과: 비유행할 것
신조어: 세상이 날 억까해 -> 예측 결과: 비유행할 것
신조어: 불소 

In [55]:
# ✅ 예측 결과 샘플링

n_samples = 5
sample_rows = predict_df.sample(n=n_samples, random_state=random.randint(0,10000))

sample_texts = sample_rows['클린 신조어'].fillna('')
sample_features = vectorizer.transform(sample_texts)
sample_preds = model.predict(sample_features)

print("\n[미래 신조어 유행 여부 예측 결과]\n")
for word, pred in zip(sample_rows['클린 신조어'], sample_preds):
    print(f"신조어: {word} -> 예측 결과: {'유행할 것' if pred == 1 else '비유행할 것'}")


[미래 신조어 유행 여부 예측 결과]

신조어: 철컹철컹 -> 예측 결과: 비유행할 것
신조어: 불소 -> 예측 결과: 유행할 것
신조어: 머선129 -> 예측 결과: 비유행할 것
신조어: 전공 -> 예측 결과: 비유행할 것
신조어: 미친년 -> 예측 결과: 비유행할 것


In [61]:
# ✅ 예측 확률 기반 임계값 설정 준비

sample_probs = model.predict_proba(sample_features)

threshold = 0.6  # 기본 0.5 → 0.6으로 실험
for word, prob in zip(sample_rows['클린 신조어'], sample_probs):
    prediction = 1 if prob[1] >= threshold else 0
    print(f"신조어: {word} -> 확률: {prob[1]:.4f} -> 예측 결과: {'유행할 것' if prediction == 1 else '비유행할 것'}")

신조어: 철컹철컹 -> 확률: 0.4982 -> 예측 결과: 비유행할 것
신조어: 불소 -> 확률: 0.5975 -> 예측 결과: 비유행할 것
신조어: 머선129 -> 확률: 0.3996 -> 예측 결과: 비유행할 것
신조어: 전공 -> 확률: 0.4982 -> 예측 결과: 비유행할 것
신조어: 미친년 -> 확률: 0.4982 -> 예측 결과: 비유행할 것


In [65]:
# ✅ 다양한 임계값 테스트

for threshold in [0.5, 0.6, 0.7]:
    print(f"\n[임계값 {threshold} 적용 결과]")
    for word, prob in zip(sample_rows['클린 신조어'], sample_probs):
        prediction = 1 if prob[1] >= threshold else 0
        print(f"신조어: {word} -> 확률: {prob[1]:.4f} -> 예측 결과: {'유행할 것' if prediction == 1 else '비유행할 것'}")


[임계값 0.5 적용 결과]
신조어: 철컹철컹 -> 확률: 0.4982 -> 예측 결과: 비유행할 것
신조어: 불소 -> 확률: 0.5975 -> 예측 결과: 유행할 것
신조어: 머선129 -> 확률: 0.3996 -> 예측 결과: 비유행할 것
신조어: 전공 -> 확률: 0.4982 -> 예측 결과: 비유행할 것
신조어: 미친년 -> 확률: 0.4982 -> 예측 결과: 비유행할 것

[임계값 0.6 적용 결과]
신조어: 철컹철컹 -> 확률: 0.4982 -> 예측 결과: 비유행할 것
신조어: 불소 -> 확률: 0.5975 -> 예측 결과: 비유행할 것
신조어: 머선129 -> 확률: 0.3996 -> 예측 결과: 비유행할 것
신조어: 전공 -> 확률: 0.4982 -> 예측 결과: 비유행할 것
신조어: 미친년 -> 확률: 0.4982 -> 예측 결과: 비유행할 것

[임계값 0.7 적용 결과]
신조어: 철컹철컹 -> 확률: 0.4982 -> 예측 결과: 비유행할 것
신조어: 불소 -> 확률: 0.5975 -> 예측 결과: 비유행할 것
신조어: 머선129 -> 확률: 0.3996 -> 예측 결과: 비유행할 것
신조어: 전공 -> 확률: 0.4982 -> 예측 결과: 비유행할 것
신조어: 미친년 -> 확률: 0.4982 -> 예측 결과: 비유행할 것


In [67]:
# ✅ 데이터 불균형 문제 발견
# 유행 신조어 수가 훨씬 많아서 로지스틱 회귀가 1(유행할 것)로 편향될 가능성 있음
# 향후 개선 방향: 언더샘플링, 오버샘플링, 또는 다른 모델 적용 고려

In [ ]:
# 1. 라이브러리 불러오기
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import re

# 2. 데이터 불러오기
df = pd.read_csv('/content/신조어_데이터셋_최종본_랜덤라벨링.csv')

# 3. 신조어 번호 제거 (예: "57.걍고" -> "걍고")
def clean_word(word):
    word = str(word)
    return re.sub(r'^\d+\.', '', word).strip()

df['클린 신조어'] = df['신조어'].apply(clean_word)

# 4. 텍스트 데이터와 레이블 준비
texts = df['설명'].fillna('')  # 설명이 없으면 빈 문자열
labels = df['유행여부']  # 0 or 1

# 5. TF-IDF 벡터화
vectorizer = TfidfVectorizer(max_features=500)  # 최대 500개 단어로 제한
X = vectorizer.fit_transform(texts)

# 6. 학습용 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

# 7. 로지스틱 회귀로 학습
model = LogisticRegression()
model.fit(X_train, y_train)

# 8. 예측 및 평가
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"\n[TF-IDF + 로지스틱 회귀] 유행 여부 예측 정확도: {accuracy:.4f}\n")
print(classification_report(y_test, y_pred, target_names=["비유행(0)", "유행(1)"]))

# 9. 예시로 5개 문장에 대해 유행 여부 예측 출력
sample_texts = df['설명'].sample(5, random_state=42)

print("\n[예시 5개 신조어 설명에 대한 유행 여부 예측]")
sample_features = vectorizer.transform(sample_texts)
sample_preds = model.predict(sample_features)

for text, pred in zip(sample_texts, sample_preds):
    print(f"설명: {text[:30]}... -> 예측: {'유행' if pred == 1 else '비유행'}")



[TF-IDF + 로지스틱 회귀] 유행 여부 예측 정확도: 0.6364

              precision    recall  f1-score   support

      비유행(0)       0.65      0.27      0.38        41
       유행(1)       0.63      0.90      0.74        58

    accuracy                           0.64        99
   macro avg       0.64      0.58      0.56        99
weighted avg       0.64      0.64      0.59        99


[예시 5개 신조어 설명에 대한 유행 여부 예측]
설명: 만나서 반가워 잘 부탁해... -> 예측: 유행
설명: 발로 컨트롤하다... -> 예측: 유행
설명: 인터넷에 있는 사진 동영상 등의 자료가 삭제되었거나 경... -> 예측: 유행
설명: 엄청나게 재미있음... -> 예측: 유행
설명: 연인이 된지 22일 되는날.... -> 예측: 유행


In [ ]:
# 1. 라이브러리 불러오기
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import re

# 2. 데이터 불러오기
df = pd.read_csv('/content/신조어_데이터셋_최종본_랜덤라벨링.csv')

# 3. 데이터 전처리: 신조어 번호 제거
def clean_word(word):
    word = str(word)
    return re.sub(r'^\d+\.', '', word).strip()

df['클린 신조어'] = df['신조어'].apply(clean_word)

# 4. Feature(설명 텍스트)와 Label(유행 여부) 준비
texts = df['설명'].fillna('')  # 설명 없는 경우 빈 문자열로 대체
labels = df['유행여부']

# 5. 벡터화 시도: TF-IDF 선택 이유
# 실험: CountVectorizer보다 TF-IDF가 빈도 중요도를 반영할 수 있어 정보량 손실을 줄일 수 있음
vectorizer = TfidfVectorizer(max_features=500)  # 최대 500개 단어로 제한
X = vectorizer.fit_transform(texts)

# 6. 데이터 분할
# 처음에는 random_state 고정해서 실험했지만, 매번 랜덤하게 성능 보는게 낫다고 판단 → random_state 제거
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2)

# 7. 모델 선택: 로지스틱 회귀
# 실험: 단순 선형 분류 문제이기 때문에 SVM보다는 빠르고 해석 가능한 Logistic Regression 선택
model = LogisticRegression()
model.fit(X_train, y_train)

# 8. 예측 및 평가
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"\n[TF-IDF + 로지스틱 회귀] 유행 여부 예측 정확도: {accuracy:.4f}\n")
print(classification_report(y_test, y_pred, target_names=["비유행(0)", "유행(1)"]))

# 9. 예시 5개 문장 예측
# 이전에는 random_state로 고정되었지만, 실험 다양성 확보를 위해 random_state 없이 random 샘플링 적용
sample_texts = df['설명'].sample(5)

print("\n[예시 5개 신조어 설명에 대한 유행 여부 예측]")
sample_features = vectorizer.transform(sample_texts)
sample_preds = model.predict(sample_features)

for text, pred in zip(sample_texts, sample_preds):
    print(f"설명: {text[:30]}... -> 예측: {'유행' if pred == 1 else '비유행'}")

# 10. 성찰 및 개선점 코멘트
print("\n[성찰 및 개선점]")
print("- 현재 TF-IDF로 설명만 벡터화했는데, 설명이 짧거나 빈약한 경우 예측 성능이 떨어질 수 있음")
print("- 유행여부를 더 잘 맞추려면 '설명 + 스타일 패턴'을 feature로 함께 넣는 확장 시도가 가능함")
print("- 또는 신조어 자체를 subword 수준으로 분석해 특징을 추가하는 것도 가능")



[TF-IDF + 로지스틱 회귀] 유행 여부 예측 정확도: 0.5455

              precision    recall  f1-score   support

      비유행(0)       0.48      0.23      0.31        44
       유행(1)       0.56      0.80      0.66        55

    accuracy                           0.55        99
   macro avg       0.52      0.51      0.48        99
weighted avg       0.53      0.55      0.50        99


[예시 5개 신조어 설명에 대한 유행 여부 예측]
설명: 미치도록 클릭... -> 예측: 유행
설명: 얼굴 0점... -> 예측: 비유행
설명: 자연스러운 만남을 추구... -> 예측: 비유행
설명: 누구 물어본 사람?... -> 예측: 비유행
설명: 무슨일이야?... -> 예측: 유행

[성찰 및 개선점]
- 현재 TF-IDF로 설명만 벡터화했는데, 설명이 짧거나 빈약한 경우 예측 성능이 떨어질 수 있음
- 유행여부를 더 잘 맞추려면 '설명 + 스타일 패턴'을 feature로 함께 넣는 확장 시도가 가능함
- 또는 신조어 자체를 subword 수준으로 분석해 특징을 추가하는 것도 가능


In [66]:
# ✅ 실험 결과 해석
# - 임계값 조정에 따라 유행/비유행 분류가 다르게 나타남
# - 기본 0.5 임계값이 가장 안정적이나, 유행 신조어 예측 Recall을 높이려면 0.6~0.7도 고려할 수 있음
# - 향후: 신조어 자체 특징(길이, 음운, 감정성 등) feature 추가 시도해볼 것